In [2]:
import datetime
import logging
import pickle
import sys
import warnings
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import pandas as pd
import polars as pl
from prophet import Prophet
from prophet.plot import plot_yearly, plot_seasonality
from sklearn.metrics import mean_absolute_error
from tqdm import tqdm

sys.path.append("..")
from src.processing import preprocessing

# https://stackoverflow.com/questions/66667909/stop-printing-infocmdstanpystart-chain-1-infocmdstanpyfinish-chain-1
logger = logging.getLogger("cmdstanpy")
logger.addHandler(logging.NullHandler())
logger.propagate = False
logger.setLevel(logging.CRITICAL)

warnings.filterwarnings("ignore", category=FutureWarning, message="The behavior of DatetimeProperties.to_pydatetime is deprecated")

DATA_DIR = Path("../data")

/Users/shu.omura/workspace/github/pjme/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [3]:
df = pl.read_csv(DATA_DIR / "PJME_hourly.csv")
split_date = datetime.datetime(2015, 1, 1)

df = preprocessing(df)
df_train, df_test = df.filter(pl.col("ds") <= split_date), df.filter(pl.col("ds") > split_date)
df, df_train, df_test = df.to_pandas(), df_train.to_pandas(), df_test.to_pandas()

## 4thモデル
- 夏場と冬場の傾向を捉える
    - `add_seasonality`でカスタムの季節変動を加える
      - `condition_name`で適用するデータポイントを絞ることができる
    - 祝日効果として夏場や冬場のピークを組み込む
    - `add_regressor`で気温などの外部要因をモデルに組み込む

### add_seasonalityの追加
`add_seasonality`は精度向上しない。夏以外の期間にも、period=92の周期変動を適用するので、ちょっとモデリングに無理があるかも

In [4]:
outputs = []
best_fourier_order, best_score = None, np.inf
for i in tqdm(range(1, 11, 2), total=len(range(1, 11, 2))):
    model = Prophet(yearly_seasonality=2, weekly_seasonality=1)
    # 6/15-9/15までの92日間をhigh_seasonとして定義
    model.add_seasonality(name="summer", period=92, fourier_order=i)
    model.fit(df_train)

    preds = model.predict(df_test)
    mae = mean_absolute_error(
        df_test["y"],
        pl.DataFrame(preds).filter(pl.col("ds") > split_date)["yhat"],
    )
    if mae < best_score:
        best_score = mae
        best_fourier_order = i
    
    outputs.append(f"weekly_seasonality={i} MAE: {mae}")  # best so far: 5100.535160661208

for output in outputs:
    print(output)
print(f"best_fourier_order: {best_fourier_order}, best_score: {best_score}")

  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 5/5 [02:51<00:00, 34.39s/it]

weekly_seasonality=1 MAE: 5110.973782590863
weekly_seasonality=3 MAE: 5105.000975367784
weekly_seasonality=5 MAE: 5105.163656682453
weekly_seasonality=7 MAE: 5105.011670019132
weekly_seasonality=9 MAE: 5107.398180108165
best_fourier_order: 3, best_score: 5105.000975367784


毎年`6/15-9/15`を夏と定義し`is_summer`フラグをデータに追加。  
`is_summer`フラグが`True`の期間に対してのみ、`add_seasonality`を追加してみる。
- 結果
  - 効果なし

In [5]:
add_summer_df = pl.DataFrame(df)
add_summer_df = add_summer_df.with_columns(
    (
        pl.col("ds").dt.month().is_in([7, 8, 9]) |
        ((pl.col("ds").dt.month() == 6) & (pl.col("ds").dt.day() >= 15)) |
        ((pl.col("ds").dt.month() == 9) & (pl.col("ds").dt.day() <= 15))
    ).cast(pl.UInt8).alias("is_summer")
)

add_summer_df_train, add_summer_df_test = add_summer_df.filter(pl.col("ds") <= split_date), add_summer_df.filter(pl.col("ds") > split_date)
add_summer_df, add_summer_df_train, add_summer_df_test = add_summer_df.to_pandas(), add_summer_df_train.to_pandas(), add_summer_df_test.to_pandas()

In [6]:
model = Prophet(yearly_seasonality=2, weekly_seasonality=1)
model.add_seasonality(name="summer", period=92, fourier_order=3, condition_name="is_summer")
model.fit(add_summer_df_train)

preds = model.predict(add_summer_df_test)

mae = mean_absolute_error(
    add_summer_df_test["y"],
    pl.DataFrame(preds).filter(pl.col('ds') > split_date)["yhat"],
)
print(f"MAE: {mae}")  # not set condition_name MAE: 5105.000975367784

MAE: 5113.1217911818


### add_regressorの追加
`add_regressor`で、シカゴの過去の気温情報を加えてみる。  
データは、[NOAA](https://www.ncdc.noaa.gov/cdo-web/datasets/GHCND/locations/CITY:US170006/detail)から取得。  
今回の電力消費データは、PJMというアメリカ東部の電力会社が集計したものであり、下記の州を管轄している。下記の週の中でもっと人口が多い都市であるオハイオ州のシカゴの気温データを使う。
```
It is part of the Eastern Interconnection grid operating an electric transmission system serving all or parts of Delaware, Illinois, Indiana, Kentucky, Maryland, Michigan, New Jersey, North Carolina, Ohio, Pennsylvania, Tennessee, Virginia, West Virginia, and the District of Columbia
```
https://www.kaggle.com/datasets/robikscube/hourly-energy-consumption

In [7]:
chicago_df = pl.read_csv(DATA_DIR / "temperature" / "chicago.csv", dtypes={"high": pl.Float32, "low": pl.Float32})
print(f"収集データ場所: {chicago_df['NAME'][0]}")

chicago_df = chicago_df.with_columns(
    pl.col("DATE").str.to_datetime(time_unit="ns").alias("ds")
).drop("NAME", "DATE")
display(chicago_df.head())

# 元データと結合
df = pl.DataFrame(df).join(chicago_df, on="ds", how="left")
df = df.with_columns(
    pl.col("high").forward_fill(),
    pl.col("low").forward_fill(),
)

収集データ場所: CHICAGO OHARE INTERNATIONAL AIRPORT, IL US


high,low,ds
f32,f32,datetime[ns]
2.5,1.0,2002-01-01 00:00:00
3.1,1.1,2002-01-02 00:00:00
2.8,1.2,2002-01-03 00:00:00
3.8,1.8,2002-01-04 00:00:00
3.5,2.8,2002-01-05 00:00:00


残念ながらスコアは向上せず。日単位で同じ気温を使っていることが問題かも。  
prophetモデルは結構繊細な印象。LightGBMはこの特徴量でスコアを向上させている。  
しっかりEDAを行って適切に要素をモデルに組み込んでいく必要がありそう。

In [12]:
model = Prophet(yearly_seasonality=2, weekly_seasonality=1)
model.add_regressor("high")
model.add_regressor("low")

df = df.drop_nulls()

df_train, df_test = df.filter(pl.col("ds") <= split_date), df.filter(pl.col("ds") > split_date)
df, df_train, df_test = df, df_train.to_pandas(), df_test.to_pandas()
model.fit(df_train)
preds = model.predict(df_test)

mae = mean_absolute_error(
    df_test["y"],
    pl.DataFrame(preds).filter(pl.col("ds") > split_date)["yhat"],
)
print(f"MAE: {mae}")

MAE: 5124.377598955317


その他、変化点の検知を調整したり、加法モデルから乗法モデルに変更したり改善の余地がある。  
ただ精度という観点ではLightGBMには遠く及ばない。しかしより直感的にモデルに情報を組み込むことができ、解釈性に富んでいるのが大きなメリット